In [1]:
import multiqc
from collections import defaultdict
from multiqc.plots import table

In [25]:
multiqc.reset()

In [2]:
multiqc.parse_logs(
    [
        "/Users/vlad/tmp/nextflow/assembly",
        "/Users/vlad/tmp/nextflow/fastp",
        "/Users/vlad/tmp/nextflow/variants",
    ],
    force=True
)
print(multiqc.list_modules())

/// https://multiqc.info 🔍 | v1.22.dev0 (8f3e39a)
|       init_config | Using log filenames for sample names
|       file_search | Search path : /Users/vlad/tmp/nextflow/assembly
|       file_search | Search path : /Users/vlad/tmp/nextflow/fastp
|       file_search | Search path : /Users/vlad/tmp/nextflow/variants
|         searching | ████████████████████████████████████████ 100% 10/10
|          cutadapt | Found 1 reports
|             fastp | Found 3 reports
['Cutadapt', 'fastp']


In [3]:
multiqc.list_data_sources()

['/Users/vlad/tmp/nextflow/assembly/cutadapt/log/SAMPLE2_PE.primer_trim.cutadapt.log',
 '/Users/vlad/tmp/nextflow/fastp/SAMPLE3_SE.fastp.json',
 '/Users/vlad/tmp/nextflow/fastp/SAMPLE1_PE.fastp.json',
 '/Users/vlad/tmp/nextflow/fastp/SAMPLE2_PE.fastp.json']

In [4]:
multiqc.parse_logs(
    "/Users/vlad/tmp/nextflow/fastqc",
    module_order=[
        dict(
            fastqc=dict(
                name="FastQC (trimmed)",
                anchor="fastqc_trimmed",
                info="This section of the report shows FastQC results after adapter trimming.",
                target="",
                path_filters=["*/trim/*"],
            )
        ),
        dict(
            fastqc=dict(
                name="FastQC (raw)",
                anchor="fastqc_raw",
                path_filters=["*/raw/*"],
            )
        ),
    ]
)
print(multiqc.list_modules())

/// https://multiqc.info 🔍 | v1.22.dev0 (8f3e39a)
|       init_config | Using log filenames for sample names
|       file_search | Search path : /Users/vlad/tmp/nextflow/fastqc
|         searching | ████████████████████████████████████████ 100% 16/16
|            fastqc | Found 3 reports
|            fastqc | Found 5 reports
['Cutadapt', 'fastp', 'FastQC (trimmed)', 'FastQC (raw)']


In [5]:
multiqc.list_data_sources()

['/Users/vlad/tmp/nextflow/assembly/cutadapt/log/SAMPLE2_PE.primer_trim.cutadapt.log',
 '/Users/vlad/tmp/nextflow/fastp/SAMPLE3_SE.fastp.json',
 '/Users/vlad/tmp/nextflow/fastp/SAMPLE1_PE.fastp.json',
 '/Users/vlad/tmp/nextflow/fastp/SAMPLE2_PE.fastp.json',
 '/Users/vlad/tmp/nextflow/fastqc/trim/SAMPLE3_SE_fastqc.zip',
 '/Users/vlad/tmp/nextflow/fastqc/trim/SAMPLE2_PE_2_fastqc.zip',
 '/Users/vlad/tmp/nextflow/fastqc/trim/SAMPLE2_PE_1_fastqc.zip',
 '/Users/vlad/tmp/nextflow/fastqc/raw/SAMPLE1_PE_1_fastqc.zip',
 '/Users/vlad/tmp/nextflow/fastqc/raw/SAMPLE3_SE_fastqc.zip',
 '/Users/vlad/tmp/nextflow/fastqc/raw/SAMPLE2_PE_2_fastqc.zip',
 '/Users/vlad/tmp/nextflow/fastqc/raw/SAMPLE2_PE_1_fastqc.zip',
 '/Users/vlad/tmp/nextflow/fastqc/raw/SAMPLE1_PE_2_fastqc.zip']

In [6]:
multiqc.parse_logs(
    "/Users/vlad/tmp/nextflow/kraken2",
    verbose=True
)
print(multiqc.list_modules())

/// https://multiqc.info 🔍 | v1.22.dev0 (8f3e39a)
[2024-04-25 16:56:35] multiqc.core.init_config                           [DEBUG  ]  This is MultiQC v1.22.dev0 (8f3e39a)
[2024-04-25 16:56:35] multiqc.core.init_config                           [DEBUG  ]  Command used: /Users/vlad/git/MultiQC/venv/lib/python3.12/site-packages/ipykernel_launcher.py -f /Users/vlad/Library/Jupyter/runtime/kernel-9368ebf4-92b2-4596-a56b-bf9595eed86a.json
[2024-04-25 16:56:35] multiqc.core.init_config                           [DEBUG  ]  Latest MultiQC version is v1.21, released 2024-02-28
[2024-04-25 16:56:35] multiqc.core.init_config                           [INFO   ]  Using log filenames for sample names
[2024-04-25 16:56:35] multiqc.core.init_config                           [DEBUG  ]  Working dir : /Users/vlad/git/MultiQC/notebooks
[2024-04-25 16:56:35] multiqc.core.init_config                           [DEBUG  ]  Template    : default
[2024-04-25 16:56:35] multiqc.core.init_config                     

Now add additional table to replace the general stats table

In [7]:
summarized_data = defaultdict(dict)
for s in multiqc.list_samples():
    if data := multiqc.get_module_data(sample=s, module="fastp"):
        summarized_data[s]["input_reads"] = data["summary"]["before_filtering"]["total_reads"]
    if (data := multiqc.get_module_data(sample=s, module="Cutadapt")) and "r_written" in data:
        summarized_data[s]["trimmed_reads"] = data["r_written"]
    if (data := multiqc.get_general_stats_data(sample=s)) and "pct_unclassified" in data:
        summarized_data[s]["non_host_reads"] = data["pct_unclassified"]
print(dict(summarized_data))

{'SAMPLE1_PE': {'input_reads': 55442}, 'SAMPLE2_PE': {'input_reads': 42962}, 'SAMPLE3_SE': {'input_reads': 49202}}


In [12]:
multiqc.add_custom_content_section(
    plot=table.plot(
        data=summarized_data,
        headers={
            "input_reads": {
                "title": "# Input reads",
                "description": "Number of reads before filtering",
                "scale": False,
                "format": "{:,.0f}",
            },
            "trimmed_reads": {
                "title": "# Trimmed reads (Cutadapt)",
                "description": "Number of reads after adapter trimming",
                "scale": False,
                "format": "{:,.0f}",
            },
            "non_host_reads": {
                "title": "% Non-host reads (Kraken 2)",
                "description": "Percentage of reads classified as non-host",
                "scale": False,
                "format": "{:.2f}",
            },
        },
        pconfig={
            "id": "summary_assembly_metrics",
            "title": "Summary Assembly Metrics",
            "description": "Summary of input reads, trimmed reads, and non-host reads.",
        },
    ),
    name="Summary Assembly Metrics",
    anchor="summary_assembly",
    description="Summary of input reads, trimmed reads, and non-host reads.",
)

In [9]:
# Now generate report from the loaded data + summarized_data
multiqc.write_report(
    excluded_modules=["general_stats"],
    force=True,
    verbose=True
)

/// https://multiqc.info 🔍 | v1.22.dev0 (8f3e39a)
[2024-04-25 16:56:49] multiqc.core.init_config                           [DEBUG  ]  This is MultiQC v1.22.dev0 (8f3e39a)
[2024-04-25 16:56:49] multiqc.core.init_config                           [DEBUG  ]  Command used: /Users/vlad/git/MultiQC/venv/lib/python3.12/site-packages/ipykernel_launcher.py -f /Users/vlad/Library/Jupyter/runtime/kernel-9368ebf4-92b2-4596-a56b-bf9595eed86a.json
[2024-04-25 16:56:49] multiqc.core.init_config                           [DEBUG  ]  Latest MultiQC version is v1.21, released 2024-02-28
[2024-04-25 16:56:49] multiqc.core.init_config                           [INFO   ]  Using log filenames for sample names
[2024-04-25 16:56:49] multiqc.core.init_config                           [DEBUG  ]  Working dir : /Users/vlad/git/MultiQC/notebooks
[2024-04-25 16:56:49] multiqc.core.init_config                           [DEBUG  ]  Template    : default
[2024-04-25 16:56:49] multiqc.core.init_config                     

In [10]:
multiqc.list_plots()

['cutadapt_filtered_reads_plot',
 'cutadapt_trimmed_sequences_plot_default (2 datasets)',
 'fastp_filtered_reads_plot',
 'fastp-insert-size-plot',
 'fastp-seq-quality-plot (4 datasets)',
 'fastp-seq-content-gc-plot (4 datasets)',
 'fastp-seq-content-n-plot (4 datasets)',
 'fastqc_sequence_counts_plot',
 'fastqc_per_base_sequence_quality_plot',
 'fastqc_per_sequence_quality_scores_plot',
 'fastqc_per_sequence_gc_content_plot (2 datasets)',
 'fastqc_per_base_n_content_plot',
 'fastqc_sequence_length_distribution_plot',
 'fastqc_sequence_duplication_levels_plot',
 'fastqc_overrepresented_sequences_plot',
 'fastqc_top_overrepresented_sequences_table',
 'fastqc-status-check-heatmap',
 'fastqc_sequence_counts_plot-1',
 'fastqc_per_base_sequence_quality_plot-1',
 'fastqc_per_sequence_quality_scores_plot-1',
 'fastqc_per_sequence_gc_content_plot-1 (2 datasets)',
 'fastqc_per_base_n_content_plot-1',
 'fastqc_sequence_length_distribution_plot-1',
 'fastqc_sequence_duplication_levels_plot-1',
 'f

In [11]:
multiqc.show_plot("summary_assembly_metrics")

KeyError: 'summary_assembly_metrics'